In [ ]:
# Centralized imports and configuration of matplotlib
# Enable interactive matplotlib backend for Jupyter
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import networkx as nx
import os
import sys
from datetime import date, timedelta
from collections import defaultdict
from matplotlib.widgets import Button
from project_utils.project import Project

# KLUDGE(RAKN): Use a global variable to allow all cells to access the same project
global project


## Enter tasks
First you have to enter the tasks or activities of the project into the `tasks` data structure below. Alternatively, you can enter them in the "Task editor" table below.

When copying tables from PDFs to a text editor like Sublime, the table gets flattened and each cell forms a separate row. To disentangle the cells and extract column by column, use `awk 'NR % 4 == 1' <input file>` to extract the 1st column, use `4 == 2` to extract the second column etc. and use `4 == 0` to extract the last column.

In [ ]:
from datetime import date

project_name = 'iDesign Normal Solution Lab Example Project'
project_start_date = date(2020, 1, 6)

tasks = {
    'Task Name': ['Requirements', 'Architecture', 'Project Planning', 'Test Plan', 'Test Harness', 'Logging', 'Security', 'Pub/Sub', 'Resource A', 'Resource B', 'Resource Access A', 'Resource Access B', 'Resource Access C', 'EngineA', 'EngineB', 'EngineC', 'ManagerA', 'ManagerB', 'Client App1', 'Client App2', 'System Testing'],
    'Duration': [15, 20, 20, 30, 35, 15, 20, 5, 20, 15, 10, 5, 15, 20, 25, 15, 20, 25, 25, 35, 30],
    'Predecessors': ['', '1', '2', '3', '4', '3', '3', '3', '3', '3', '6,9', '6,10', '6', '12,13', '12,13', '6', '7,8,11,14,15', '7,8,15,16', '17,18', '17', '5,19,20']
}

assert len(tasks['Task Name']) == len(tasks['Duration']) == len(tasks['Predecessors'])

## Task editor
After having executed the cell below, you can find a table with any tasks defined above. The table is editable and the changes are taken into account in the following project design steps.

How to use the task editor:
- Edit tasks by double-clicking cells in the grid
- Add new tasks with the '➕ Add New Task' button
- Remove the last task with the '🗑️ Remove Last Task' button
- Use toolbar buttons for sorting, filtering, and more
- Preview changes before applying
- Reset to original values anytime
- Click 'Apply Changes & Create Project' when ready!

In [ ]:
import qgridnext as qgrid
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Create DataFrame
task_df = pd.DataFrame(tasks)

# Create qgrid widget for interactive editing
qgrid_widget = qgrid.show_grid(
    task_df, 
    show_toolbar=True,
    grid_options={
        'syncColumnCellResize': True,
        'forceFitColumns': False,
        'defaultColumnWidth': 150,
        'enableColumnReorder': False,
        'enableTextSelectionOnCells': True,
        'rowHeight': 28,
        'enableCellNavigation': True,
        'fullWidthRows': True
    },
    column_options={
        'editable': True
    },
    column_definitions={
        'Task Name': {'width': 200, 'editable': True},
        'Duration': {'width': 100, 'editable': True},
        'Predecessors': {'width': 150, 'editable': True}
    }
)

# Output area for messages and results
output_area = widgets.Output()

# Function to create project from qgrid data
def create_project_from_qgrid():
    """Create a project from the qgrid data and return it."""
    # Get the current data from qgrid
    current_data = qgrid_widget.get_changed_df()
    
    # Create new project
    project = Project(project_name, project_start_date)
    
    # Add tasks from the grid data
    for idx, row in current_data.iterrows():
        task_name = row['Task Name']
        duration = int(row['Duration'])
        predecessors_str = str(row['Predecessors']).strip()
        
        if predecessors_str and predecessors_str != 'nan':
            # Parse predecessors (comma-separated string)
            pred_list = [int(p.strip()) for p in predecessors_str.split(',') if p.strip().isdigit()]
            project.add_task(task_name, duration, predecessors=pred_list)
        else:
            project.add_task(task_name, duration)
    
    return project

# Function to add a new row to the qgrid
def add_new_task():
    """Add a new task row to the qgrid."""
    with output_area:
        output_area.clear_output()
        try:
            current_data = qgrid_widget.get_changed_df()
            next_task_num = len(current_data) + 1
            
            # Create new row
            new_row = pd.DataFrame({
                'Task Name': [f'Task {next_task_num}'],
                'Duration': [0],
                'Predecessors': ['']
            })
            
            # Add to existing data
            updated_data = pd.concat([current_data, new_row], ignore_index=True)
            
            # Update the qgrid with new data
            qgrid_widget.df = updated_data
            
            print(f"➕ Added Task {next_task_num}")
        except Exception as e:
            print(f"❌ Error adding task: {e}")

# Function to remove the last row from qgrid
def remove_last_task():
    """Remove the last task row from the qgrid."""
    with output_area:
        output_area.clear_output()
        try:
            current_data = qgrid_widget.get_changed_df()
            if len(current_data) <= 1:
                print("❌ Cannot remove the last task. At least one task is required.")
                return
            
            # Remove last row
            updated_data = current_data.iloc[:-1].copy()
            
            # Update the qgrid
            qgrid_widget.df = updated_data
            
            print(f"🗑️ Removed last task")
        except Exception as e:
            print(f"❌ Error removing task: {e}")

# Function to reset to original data
def reset_to_default():
    """Reset qgrid to original task data."""
    with output_area:
        output_area.clear_output()
        qgrid_widget.df = pd.DataFrame(tasks)
        print("🔄 Reset to default values")

# Function to preview current data
def preview_data():
    """Preview current qgrid data."""
    with output_area:
        output_area.clear_output()
        current_data = qgrid_widget.get_changed_df()
        print("📋 Current Task Data:")
        display(current_data)

# Event handler for apply changes
def on_apply_changes(b):
    with output_area:
        output_area.clear_output()
        try:
            global project
            project = create_project_from_qgrid()
            print(f"✅ Project updated: {project.name}")
            print(f"📊 Tasks: {len(project.tasks)}")
            
            # Display updated task schedule
            task_schedule = project.schedule_tasks()
            display(task_schedule)
        except Exception as e:
            print(f"❌ Error: {e}")

# Create buttons
add_task_button = widgets.Button(description="➕ Add New Task", button_style='primary')
remove_task_button = widgets.Button(description="🗑️ Remove Last Task", button_style='warning')
apply_button = widgets.Button(description="Apply Changes & Create Project", button_style='success')
reset_button = widgets.Button(description="Reset to Default", button_style='warning')
preview_button = widgets.Button(description="Preview Data", button_style='info')

# Connect event handlers
add_task_button.on_click(lambda b: add_new_task())
remove_task_button.on_click(lambda b: remove_last_task())
apply_button.on_click(on_apply_changes)
reset_button.on_click(lambda b: reset_to_default())
preview_button.on_click(lambda b: preview_data())

# Organize buttons
task_management_buttons = widgets.HBox([add_task_button, remove_task_button])
main_buttons = widgets.HBox([apply_button, reset_button, preview_button])

on_apply_changes(apply_button)

# Display the interface
display(qgrid_widget)
display(task_management_buttons)
display(main_buttons)
# display(output_area)


In [ ]:
# The following code is a backup approach should the interactive approach above not work.

# # Create and set up the project
# project = Project("iDesign Lab Example Project", date(2020, 1, 6))

# # Add tasks
# project.add_task("Task 1", 10)
# project.add_task("Task 2", 20, predecessors=[1])
# project.add_task("Task 3", 40, predecessors=[1])
# project.add_task("Task 4", 30, predecessors=[1])
# project.add_task("Task 5", 10, predecessors=[2])
# project.add_task("Task 6", 0, predecessors=[3])
# project.add_task("Task 7", 10, predecessors=[3])
# project.add_task("Task 8", 30, predecessors=[5])
# project.add_task("Task 9", 20, predecessors=[6, 8])
# project.add_task("Task 10", 25, predecessors=[6, 8])
# project.add_task("Task 11", 10, predecessors=[4, 7, 9])
# project.add_task("Task 12", 10, predecessors=[10])
# project.add_task("Task 13", 0, predecessors=[11, 12])
# project.add_task("Task 14", 10, predecessors=[11, 12])
# project.add_task("Task 15", 5, predecessors=[11, 12])
# project.add_task("Task 16", 5, predecessors=[13])
# project.add_task("Task 17", 5, predecessors=[14, 16])
# project.add_task("Task 18", 5, predecessors=[15, 17])

print(f"Project: {project.name}")
print(f"Start date: {project.start_date.strftime('%A, %B %d, %Y')}")
print(f"Tasks: {len(project.tasks)}")
task_df = project.schedule_tasks()
task_df


## Plot the tasks and their dependencies
We use the arrow diagram style, where edges represent tasks and nodes represent start and/or end events of tasks.

While the node diagram style (where nodes represent tasks and edges represent inverted dependencies) is more intuitive, it falls short in the not uncommon situation of fan-in followed by a fan-out.

A heuristic generates a diagram by aligning the critical path along the vertical center as good as possible and then adding additional paths around it without nodes or edges overlapping. Because the heuristic is not perfect in avoiding crossing edges, the graph is editable. Simply drag nodes around until you are satisfied with the layout.

In [ ]:
# Generate the AoA network with interactive capability
def setup_interactive_aoa_plot():
    """Set up interactive AoA plotting in Jupyter."""
    
    # Generate the plot (this populates the aoa_*_artists attributes)
    g = project.draw_aoa_network_float_based(title=f"Interactive AoA Network - {project.name}")
    
    # Get current figure and axis
    fig = plt.gcf()
    ax = plt.gca()
    
    # Set up node tracking for interactivity
    node_order = getattr(project, 'aoa_node_order', list(g.nodes()))
    node_collection = None
    node_positions = {}
    node_indices = {}
    node_labels = {}  # Store references to node label text artists

    # Build correct mapping from nodes to their positions and indices
    for idx, node in enumerate(node_order):
        if node in project.aoa_node_artists:
            artist, pos = project.aoa_node_artists[node]
            node_collection = artist  # All nodes share the same PathCollection
            node_positions[node] = pos
            node_indices[node] = idx

    # Find and store node label text artists
    for text_artist in ax.texts:
        # Node labels have format like "E0" or similar
        if hasattr(text_artist, 'get_text'):
            text_content = text_artist.get_text()
            if text_content.startswith('E'):
                try:
                    node_num = int(text_content[1:])  # Remove 'E' and get number
                    if node_num in node_positions:
                        node_labels[node_num] = text_artist
                except (ValueError, IndexError):
                    pass

    edge_artists = getattr(project, 'aoa_edge_artists', {})
    label_artists = getattr(project, 'aoa_label_artists', {})

    dragging = {'node': None, 'offset': (0, 0)}

    def hit_test(event):
        if event.inaxes != ax or event.xdata is None or event.ydata is None:
            return None
        
        tol = 0.8  # Tolerance for clicking
        closest_node = None
        closest_distance = float('inf')
        
        # Find the closest node within tolerance
        for node, (x, y) in node_positions.items():
            distance = np.hypot(event.xdata - x, event.ydata - y)
            if distance < tol and distance < closest_distance:
                closest_distance = distance
                closest_node = node
        
        return closest_node

    def on_press(event):
        node = hit_test(event)
        if node is not None:
            dragging['node'] = node
            x, y = node_positions[node]
            dragging['offset'] = (event.xdata - x, event.ydata - y)
            print(f"Grabbed node E{node} at position ({x:.2f}, {y:.2f})")

    def on_motion(event):
        node = dragging['node']
        if node is not None and event.inaxes == ax and event.xdata is not None and event.ydata is not None:
            new_x = event.xdata - dragging['offset'][0]
            new_y = event.ydata - dragging['offset'][1]
            node_positions[node] = (new_x, new_y)
            
            # Update PathCollection offsets
            if node in node_indices and node_collection is not None:
                idx = node_indices[node]
                offsets = node_collection.get_offsets()
                offsets[idx] = [new_x, new_y]
                node_collection.set_offsets(offsets)
            
            # Update node label position
            if node in node_labels:
                node_labels[node].set_position((new_x, new_y))
            
            # Update connected edges and labels
            for (u, v), edge_artist in edge_artists.items():
                if u == node or v == node:
                    x1, y1 = node_positions[u]
                    x2, y2 = node_positions[v]
                    # Handle FancyArrowPatch objects
                    if hasattr(edge_artist, 'set_positions'):
                        edge_artist.set_positions((x1, y1), (x2, y2))
                    elif hasattr(edge_artist, 'set_data'):
                        edge_artist.set_data([x1, x2], [y1, y2])
                    else:
                        # For FancyArrowPatch, update the path
                        edge_artist.set_position_a((x1, y1))
                        edge_artist.set_position_b((x2, y2))
            
            for (u, v), label_artist in label_artists.items():
                if u == node or v == node:
                    x1, y1 = node_positions[u]
                    x2, y2 = node_positions[v]
                    label_x = (x1 + x2) / 2
                    label_y = (y1 + y2) / 2
                    label_artist.set_position((label_x, label_y))
            
            fig.canvas.draw_idle()

    def on_release(event):
        node = dragging['node']
        if node is not None:
            print(f"Released node E{node}")
            dragging['node'] = None

    def save_layout(event=None):
        """Save current node positions to JSON file."""
        layout = {int(node): {'x': float(pos[0]), 'y': float(pos[1])} 
                 for node, pos in node_positions.items()}
        with open('aoa_layout.json', 'w') as f:
            json.dump(layout, f, indent=2)
        print("Layout saved to aoa_layout.json")

    def on_key(event):
        """Handle keyboard shortcuts."""
        if event.key == 's':
            save_layout()

    # Connect event handlers
    fig.canvas.mpl_connect('button_press_event', on_press)
    fig.canvas.mpl_connect('motion_notify_event', on_motion)
    fig.canvas.mpl_connect('button_release_event', on_release)
    fig.canvas.mpl_connect('key_press_event', on_key)

    # Add save button
    button_ax = fig.add_axes([0.85, 0.01, 0.12, 0.05])
    save_button = Button(button_ax, 'Save Layout')
    save_button.on_clicked(save_layout)

    print('Interactive AoA editing enabled in Jupyter!')
    print('- Drag nodes to move them')
    print('- Press "s" to save layout')
    print('- Click "Save Layout" button to save')
    
    return fig, ax

# Call the setup function
fig, ax = setup_interactive_aoa_plot()


## Staffing requirements
- A project manager, a product owner and an architect form the core team and is available throughout the entire project.
- Each service/task requires a single person. No two persons ever work on the same service/task.
- For the first project design iteration assume that every person will be available on demand, even though in reality staffing is not that elastic.
- A person can leave the project, but not join again to avoid on-and-off scenarios.
- The initial project design iteration can use unlimited resources.
- For specialist tasks like database design start with the assumption that enough specialists available (e.g. DBAs) and create variants of the project design by substituting specialists by generalists (i.e. developers) one by one, assuming that the outcome created by generalists is usually not as well designed as by a specialist and might take longer. Compare the variants to see whether it makes a significant difference.
- Assign the best developer to the critical path, the second best to the second-most critical etc.
- ... (add your specific staffing requirements)

## Staffing editor
In the following cell you can configure some staffing aspects of your project.

In [ ]:
# Mapping of resource types to staffing phases
# ============================================

# To do:
# - adapt the staffing phases to your project
# - adapt the mapping of resource types to staffing phases to your project

# Notes:
# - DevOps is also known as Configuration Manager (CM)
# - Not all staffing resources are required at all phases of the project.

# FIXME(RAKN): use 'Planning' instead of core_team_resource_types
unlimited_staffing_phases = {
    # the fuzzy project front-end
    'Planning': {'Architect', 'Project Manager', 'Product Owner'},
    # foundational work, i.e. non-business logic services and utilities like logging, security, pub/sub etc.
    'Infrastructure': {'Architect', 'Project Manager', 'Product Owner', 'Developer', 'DevOps'},
    # the business logic services
    'Services': {'Architect', 'Project Manager', 'Product Owner', 'Developer', 'DevOps Engineer', 'Tester'},
    # verification and validation
    'Testing': {'Architect', 'Project Manager', 'Product Owner', 'DevOps Engineer', 'Tester'},
}

# The following resource name stems are used to create unique resource names.
resource_name_stems = {
    'Architect': 'Arch',
    'Project Manager': 'PM',
    'Product Owner': 'PO',
    'Developer': 'Dev',
    'DevOps Engineer': 'DevOps',
    'Test Engineer': 'Test',
    'DB Architect': 'DBA',
    'Quality Control Tester': 'QC',
}


# Core team resource types (always available throughout project)
# ==============================================================

# To do:
# - adapt the core team staffing to your project requirements

# Notes:
# - Core team members provide ongoing oversight, guidance, and coordination
# - They ensure project continuity and are available for ad-hoc decisions and support
# - Adjust counts based on project size and complexity

core_team_resource_types = {
    'Architect': 1,           # 1 architect always available for technical oversight and guidance
    'Project Manager': 1,     # 1 PM always available for coordination and stakeholder management
    'Product Owner': 1,       # 1 PO always available for requirements clarification and decisions
    # 'DevOps Engineer': 1,   # Uncomment if continuous DevOps oversight needed throughout project
}


# Mapping of tasks to resource types
# ==================================

# To do:
# - adapt the mapping of tasks to resource types to your project

# len(tasks['Task Name'])
who_does_what = {
    'Requirements': 'Architect',
    'Architecture': 'Architect',
    'Project Planning': 'Architect',
    'Test Plan': 'Test Engineer',
    'Test Harness': 'Test Engineer',
    'Logging': 'Developer',
    'Security': 'Developer',
    'Pub/Sub': 'Developer',
    'Resource A': 'DB Architect',
    'Resource B': 'DB Architect',
    'Resource Access A': 'Developer',
    'Resource Access B': 'Developer',
    'Resource Access C': 'Developer',
    'EngineA': 'Developer',
    'EngineB': 'Developer',
    'EngineC': 'Developer',
    'ManagerA': 'Developer',
    'ManagerB': 'Developer',
    'Client App1': 'Developer',
    'Client App2': 'Developer',
    'System Testing': 'Quality Control Tester',
}

assert len(tasks['Task Name']) == len(who_does_what)

## Planned earned value
The following cells calculate the planned earned value. The resulting plot should be a "shallow S-curve". Any other shape like vertical jumps of rather flat sections are smells for unripe project design.

In [ ]:
# Planned Earned Value Analysis
# =============================

task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

person_assignments = project.assign_tasks_to_persons(
    scheduled_tasks, 
    who_does_what, 
    resource_name_stems
)

earned_value_data = project.calculate_planned_earned_value(scheduled_tasks, person_assignments)
total_duration = earned_value_data['total_duration']
enhanced_tasks = earned_value_data['sorted_tasks']
summary_stats = earned_value_data['summary_stats']

print(f"{'Task ID':<10} {'Owner':<12} {'Task Name':<25} {'Duration':<10} {'Completion':<12} {'Earned Value':<12}")
print("-" * 80)

for task_data in enhanced_tasks:
    short_task_name = task_data['name'][:23] + '..' if len(task_data['name']) > 25 else task_data['name']
    short_owner = task_data['owner'][:10] + '..' if len(task_data['owner']) > 12 else task_data['owner']
    completion_str = str(task_data['completion_date'])[:10] if task_data['completion_date'] != 'N/A' else 'N/A'
    
    print(f"{task_data['task_id']:<10} {short_owner:<12} {short_task_name:<25} {task_data['duration']:<10} {completion_str:<12} {task_data['earned_value']:>10.1f}%")

# Display summary statistics
print(f"\nSummary Statistics:")
print(f"   • Total Project Duration: {total_duration} days")
print(f"   • Total Tasks: {summary_stats['total_tasks']}")
print(f"   • Critical Path Tasks: {summary_stats['critical_path_tasks']}")
print(f"   • Average Task Duration: {summary_stats['average_task_duration']:.1f} days")
print(f"   • Total People Assigned: {summary_stats['total_people_assigned']}")


In [ ]:
# Planned Earned Value Chart
# ==========================

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd

# Get the earned value data from the previous cell's calculation
# (We'll recalculate to ensure we have the data available)
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

person_assignments = project.assign_tasks_to_persons(
    scheduled_tasks, 
    who_does_what, 
    resource_name_stems
)

earned_value_data = project.calculate_planned_earned_value(scheduled_tasks, person_assignments)
enhanced_tasks = earned_value_data['sorted_tasks']
total_duration = earned_value_data['total_duration']

# Prepare data for plotting
dates = []
earned_values = []
task_names = []

for task_data in enhanced_tasks:
    completion_date = task_data['completion_date']
    
    # Handle date conversion
    if completion_date != 'N/A' and completion_date is not None:
        try:
            # Try to convert to datetime if it's not already
            if isinstance(completion_date, str):
                # Assuming date format like '2024-01-15' or similar
                date_obj = pd.to_datetime(completion_date).date()
            else:
                date_obj = completion_date
            
            dates.append(date_obj)
            earned_values.append(task_data['earned_value'])
            task_names.append(task_data['name'][:20] + '..' if len(task_data['name']) > 20 else task_data['name'])
        except:
            # Skip tasks with invalid dates
            continue

# Create the plot in Excel "line with markers" style
plt.figure(figsize=(14, 8))

# Plot the line with markers (Excel style)
plt.plot(dates, earned_values, 
         marker='o',           # Circle markers
         markersize=6,         # Marker size
         markerfacecolor='#4472C4',  # Excel blue
         markeredgecolor='#4472C4',
         markeredgewidth=1,
         linewidth=2.5,        # Line thickness
         color='#4472C4',      # Excel blue color
         linestyle='-',        # Solid line
         alpha=0.9)

# Excel-style formatting
plt.grid(True, linestyle=':', alpha=0.6, color='gray')
plt.gca().set_facecolor('#FFFFFF')  # White background
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('#CCCCCC')
plt.gca().spines['bottom'].set_color('#CCCCCC')

# Formatting axes
plt.xlabel('Project Timeline (Completion Dates)', fontsize=12, fontweight='bold')
plt.ylabel('Planned Earned Value (%)', fontsize=12, fontweight='bold')
plt.title('Planned Earned Value Over Time\n(Cumulative Progress by Task Completion)', 
          fontsize=14, fontweight='bold', pad=20)

# Format y-axis to show percentages
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.1f}%'))
plt.ylim(0, 105)  # Start from 0% and give a bit of space above 100%

# Format x-axis for dates
if dates:
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
    plt.xticks(rotation=45, ha='right')

# Tight layout and display
plt.tight_layout()
plt.show()


## Staffing distribution chart
The following cell generates a diagram showing the staffing over the entire project duration.

Once the fuzzy front end phase is complete, the staffing should rise by the intial developers, then gradually increase to the maximum team size and towards the end the team size should gradually decline. Major jumps or falling followed by rising team sizes are smells of unripe project designs.

In [ ]:
# Staffing Distribution
# =====================

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Get the scheduled task data and assignments
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

person_assignments = project.assign_tasks_to_persons(
    scheduled_tasks, 
    who_does_what, 
    resource_name_stems
)

# Calculate staffing distribution using the NEW Project class method with core team
staffing_data = project.calculate_staffing_distribution(
    scheduled_tasks, 
    person_assignments, 
    who_does_what,
    core_team_resource_types  # Include core team configuration
)

dates = staffing_data['dates']
resource_types = staffing_data['resource_types']
staffing_matrix = staffing_data['staffing_matrix']
person_to_resource = staffing_data['person_to_resource']
core_team_counts = staffing_data['core_team_counts']

# Create header
header = f"{'Date':<12}"
for resource_type in resource_types:
    header += f"{resource_type[:10]:<12}"
header += f"{'Total':<8} {'Core Team':<10}"
print(header)
print("-" * len(header))

# Print data rows
for date in dates:
    row = f"{str(date):<12}"
    total_count = 0
    core_team_total = 0
    
    for resource_type in resource_types:
        count = staffing_matrix[date][resource_type]
        row += f"{count:<12}"
        total_count += count
        core_team_total += core_team_counts.get(resource_type, 0)
    
    row += f"{total_count:<8} {core_team_total:<10}"
    print(row)

# Prepare data for stacked bar chart
chart_data = {}
for resource_type in resource_types:
    chart_data[resource_type] = [staffing_matrix[date][resource_type] for date in dates]

# Set up the plot with Excel styling
fig, ax = plt.subplots(figsize=(15, 8))

# Excel-style colors for different resource types
excel_colors = [
    '#4472C4',  # Blue
    '#E67E22',  # Orange  
    '#70AD47',  # Green
    '#A64AC9',  # Purple
    '#F1C232',  # Yellow
    '#CE6363',  # Red
    '#4BACC6',  # Light Blue
    '#9BBB59'   # Light Green
]

# Create the stacked bar chart with core team at bottom
bottom = np.zeros(len(dates))
bars = []

# PHASE 1: Plot ALL core team members at the bottom (solid colors)
for i, resource_type in enumerate(resource_types):
    color = excel_colors[i % len(excel_colors)]
    core_counts = [core_team_counts.get(resource_type, 0) for _ in dates]
    
    # Plot core team with solid color (no hatching)
    if any(c > 0 for c in core_counts):
        core_bar = ax.bar(range(len(dates)), core_counts, 
                         bottom=bottom, label=f"{resource_type} (Core)", 
                         color=color, alpha=1.0, edgecolor='white', linewidth=0.5)
        bars.append(core_bar)
        bottom += np.array(core_counts)

# PHASE 2: Plot ALL task-based resources on top (lighter colors)
for i, resource_type in enumerate(resource_types):
    color = excel_colors[i % len(excel_colors)]
    core_counts = [core_team_counts.get(resource_type, 0) for _ in dates]
    additional_counts = [max(0, chart_data[resource_type][j] - core_counts[j]) for j in range(len(dates))]
    
    # Plot additional resources with lighter shade
    if any(c > 0 for c in additional_counts):
        add_bar = ax.bar(range(len(dates)), additional_counts, 
                        bottom=bottom, label=f"{resource_type} (Task-based)", 
                        color=color, alpha=0.6, edgecolor='white', linewidth=0.5)
        bars.append(add_bar)
        bottom += np.array(additional_counts)

# Excel-style formatting
ax.set_facecolor('#FFFFFF')  # White background
ax.grid(True, linestyle=':', alpha=0.6, color='gray', axis='y')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('#CCCCCC')
ax.spines['bottom'].set_color('#CCCCCC')

# Set labels and title
ax.set_xlabel('Project Timeline', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Resources', fontsize=12, fontweight='bold')
ax.set_title('Enhanced Staffing Distribution Over Time\n(Core Team + Task-based Resources)', 
             fontsize=14, fontweight='bold', pad=20)

# Format x-axis with dates
date_labels = [str(date) for date in dates]
ax.set_xticks(range(len(dates)))
ax.set_xticklabels(date_labels, rotation=45, ha='right', fontsize=10)

# Add legend
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)

# Set y-axis to show integers only
ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))

# Tight layout and display
plt.tight_layout()
plt.show()


## Project metrics
The following project metrics allow the comparison of different project designs.

In [ ]:
# Project Metrics Summary Table
# =============================

print("Project Metrics Summary")
print("=" * 40)

# Get the scheduled task data and assignments
task_schedule = project.schedule_tasks()
scheduled_tasks = task_schedule.to_dict('records')

person_assignments = project.assign_tasks_to_persons(
    scheduled_tasks, 
    who_does_what, 
    resource_name_stems
)

# Calculate comprehensive project metrics using the Project class method
metrics = project.calculate_project_metrics(
    scheduled_tasks,
    person_assignments,
    who_does_what,
    core_team_resource_types
)

# Display the metrics table
print(f"\n📊 PROJECT METRICS TABLE")
print("=" * 60)
print(f"{'Metric':<25} {'Value':<15} {'Unit':<15} {'Details'}")
print("-" * 60)

# a) Cost in man months (1 decimal place)
print(f"{'Cost in man months':<25} {metrics['cost_man_months']:>10.1f} {'MM':<15} Total actual effort")

# b) Duration in months (1 decimal place)
print(f"{'Duration in months':<25} {metrics['duration_months']:>10.1f} {'months':<15} Project timeline")

# c) Average staffing (1 decimal place)
print(f"{'Average staffing':<25} {metrics['average_staffing']:>10.1f} {'people':<15} Avg team size")

# d) Effort in estimated MM (integer)
print(f"{'Effort in estimated MM':<25} {metrics['estimated_effort_mm']:>10.0f} {'MM':<15} Sum of task estimates")

# e) Efficiency (integer)
print(f"{'Efficiency':<25} {metrics['efficiency_percent']:>10.0f} {'%':<15} Estimated/Actual ratio")

# f) Front end (integer)
print(f"{'Front end':<25} {metrics['frontend_ratio_percent']:>10.0f} {'%':<15} Architect-only phase")

print("-" * 60)
